In this file, we round 11 models tested in PolyRound paper and additional 12 models from bio and LP, which we will transform into matrix files later.

In [1]:
import os
from timeit import default_timer as timer
from PolyRound.api import PolyRoundApi
from PolyRound.static_classes.lp_utils import ChebyshevFinder
from PolyRound.settings import PolyRoundSettings
from PolyRound.mutable_classes.polytope import Polytope
from pathlib import Path
from os import listdir
from os.path import isfile, join
import scipy.io
import pandas as pd
import mat73

In [ ]:
settings = PolyRoundSettings()
models_path = os.path.join("PolyRound", "models")
models_path2 = os.path.join("PolyRound", "modelsMat")

onlyfiles = [f for f in listdir(models_path) if isfile(join(models_path, f)) and f[0]=='p']
onlyfolders = [f for f in listdir(models_path2) if f[0]=='p']

onlyfiles.sort()
onlyfolders.sort()

rTimes = [0]*(len(onlyfiles)+len(onlyfolders))

In [ ]:
for model in onlyfiles:
    #print(model, " started")
    model_path = os.path.join(models_path, model)
    polytope = PolyRoundApi.sbml_to_polytope(model_path) # load a model
    
    start = timer()
    one_step_rounded_polytope = PolyRoundApi.simplify_transform_and_round(polytope) # round it
    end = timer()
    
    if len(one_step_rounded_polytope.A.columns) < 1100:
        out_csv_dir = os.path.join("PolyRound", 'output', model)
        Path(out_csv_dir).mkdir(parents=True, exist_ok=True)
        PolyRoundApi.polytope_to_csvs(one_step_rounded_polytope, out_csv_dir)
    print(model, "done in", end-start, "seconds")
    rTimes[int(model[1:3])-1] = end-start

In [ ]:
for model in onlyfolders[1:2]:
    modelFolder_path = os.path.join(models_path2, model)
    Apath = join(modelFolder_path, model + '_A.mat')
    bpath = join(modelFolder_path, model + '_b.mat')
    Spath = join(modelFolder_path, model + '_S.mat')
    hpath = join(modelFolder_path, model + '_h.mat')
    ind = 0
    
    if os.path.isfile(Apath):
        M = mat73.loadmat(Apath)
        A = pd.DataFrame(M['A'])
    if os.path.isfile(bpath):
        M = scipy.io.loadmat(bpath)
        b = pd.Series(M['b'].flatten())
    if os.path.isfile(Spath):
        M = scipy.io.loadmat(Spath)
        S = pd.DataFrame(M['S'])
    if os.path.isfile(hpath):
        M = scipy.io.loadmat(hpath)
        h = pd.Series(M['h'].flatten())
        ind = 1
    
    if ind == 1:
        polytope = Polytope(A, b, S, h)
    else:
        polytope = Polytope(A, b, S)
    
    start = timer()
    one_step_rounded_polytope = PolyRoundApi.simplify_transform_and_round(polytope) # round it
    end = timer()
    
    if len(one_step_rounded_polytope.A.columns) < 1100:
        out_csv_dir = os.path.join("PolyRound", 'output', model)
        Path(out_csv_dir).mkdir(parents=True, exist_ok=True)
        PolyRoundApi.polytope_to_csvs(one_step_rounded_polytope, out_csv_dir)
    print(model, "done in", end-start, "seconds")
    rTimes[int(model[1:3])-1] = end-start

In [ ]:
# Save rounding time
rTimes = pd.DataFrame(rTimes)
rTimes.to_csv(os.path.join("PolyRound", 'output', 'roundTimes.csv')) 